### Урок 1. Введение, примеры задач, бизнес- и ML-метрики

In [1]:
import pandas as pd
import numpy as np

Представим, что в магазине всего 10 товаров

In [2]:
recommended_list = [143, 523, 11341, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [146, 52311, 16341, 3345, 1, 2]

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [3]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


#   сделать в домашней работе
def hit_rate_at_k(recommended_list, bought_list, k):
    assert k <= len(recommended_list), 'Рекомендованый лист меньше, чем заданное топ-k рекомендованных'
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [4]:
hit_rate(recommended_list, bought_list)

1

In [5]:
hit_rate_at_k(recommended_list, bought_list, k=10)

1

In [6]:
# релевантный находится на 7 позиции в recommended_list
hit_rate_at_k(recommended_list, bought_list, k=5)

0

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [7]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 992] #id товаров
bought_list = [1450, 1444, 1433, 992, 27]

In [8]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

# сделать дома
# Лучше считать через скалярное произведение, а не цикл
def money_precision_at_k(recommended_list, bought_list, prices_recommended: list, k):

    assert len(prices_recommended) >= len(bought_list), 'Должна быть указана цена в соотв. с покупками.'
    assert len(prices_recommended) >= k, 'Не все ценники указаны'
    
    relevant = np.array(list(map(lambda x: int(x in np.array(bought_list)), recommended_list)))[:k]
    print(relevant)
    
    one = (relevant * np.array(prices_recommended[:k])).sum()
    print(relevant * np.array(prices_recommended[:k]))
    
    two = (np.ones(len(prices_recommended[:k])) * np.array(prices_recommended[:k])).sum()
    print(np.ones(len(prices_recommended[:k])) * np.array(prices_recommended[:k]))
    
    precision = round((one / two) * 100, 3)
    
    return precision

In [9]:
precision(recommended_list, bought_list)

0.2

In [10]:
precision_at_k(recommended_list, bought_list, k=5)

0.2

In [11]:
money_precision_at_k(recommended_list, bought_list, [400, 60, 40, 40, 530, 33, 55, 12, 33, 77], 9)

[0 0 0 0 1 0 0 0 0]
[  0   0   0   0 530   0   0   0   0]
[400.  60.  40.  40. 530.  33.  55.  12.  33.]


44.057

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [12]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

# сделать дома
def recall_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    return recall

# сделать дома
def money_recall_at_k(recommended_list, bought_list, prices_recommended: list, prices_bought: list, k=5):
    
#     assert len(prices_recommended) >= len(bought_list), 'Должна быть указана цена в соотв. с покупками.'
#     assert len(prices_recommended) >= k, 'Не все ценники указаны'
    
    relevant = np.array(list(map(lambda x: int(x in np.array(bought_list)), recommended_list)))[:k]
    print(relevant)
    
    one = (relevant * np.array(prices_recommended[:k])).sum()
    print(relevant * np.array(prices_recommended[:k]))
    
    two =  np.array(prices_bought).sum()
    
    print(np.array(prices_bought).sum())
    
    recall = round((one / two) * 100, 3)
    
    return recall

In [13]:
recall(recommended_list, bought_list)

0.4

In [14]:
recall_at_k(recommended_list, bought_list, 9)

0.2

In [15]:
money_recall_at_k(recommended_list, bought_list, [400, 60, 40, 40, 50, 33, 55, 12, 33, 7], [45, 63, 41, 7, 50], k=10)

[0 0 0 0 1 0 0 0 0 1]
[ 0  0  0  0 50  0  0  0  0  7]
206


27.67

---

# Метрики ранжирования

Metrics: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Если важен порядок рекомендаций.

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [16]:
recommended_list = [221,2,3234,1,234,234,234,666] #id товаров
bought_list = [1,2,3,4,5,6,7,8,9]

In [17]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
  
    

In [18]:
ap_k(recommended_list, bought_list, k=5)

0.5

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [19]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]
                      ] # юзер 3

In [20]:
# сделать дома
def map_k(recommended_list, bought_list, k):
    
    list_of_ap_k = []
    for recommended_list, bought_list in zip(recommended_list, bought_list):
        list_of_ap_k.append(ap_k(recommended_list, bought_list, k))
    
    print(list_of_ap_k)
    result = np.array(list_of_ap_k).mean()
    
    return result

In [21]:
map_k(recommended_list_3_users, bought_list_3_users, k=10)

[1.0, 0, 0]


0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


In [ ]:
1, 0, 0, 1, 0 

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [ ]:
# по желанию
def ndcg_at_k():
    pass

In [ ]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 1]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [ ]:
[1, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
[143, 156, 991, 43, 11]

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [22]:
# сделать дома

def reciprocal_rank(recommended_list, bought_list, k):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    sum_ku = 0

    if k > len(recommended_list[0]):
        k = len(recommended_list[0])
    
    n_users = len(recommended_list)
    if n_users == 0:
        return 0
    
    for user in range(n_users):
        try:
            ku = np.nonzero(np.isin(recommended_list[user][:k], bought_list[user]))[0][0] + 1
            sum_ku += 1 / ku
        except IndexError:
            continue

    return sum_ku/n_users

In [23]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333


**2) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

Метрики на основе ранговой корреляции

Ранговый коэффициент корреляции Кендэлла - мера линейной связи между случайными величинами. Корреляция Кенделла является ранговой, то есть для оценки силы связи используются не численные значения, а соответствующие им ранги. Характеризует общий характер нелинейной зависимости: возрастание или убывание результативного признака при возрастании факторного. Это показатель тесноты монотонной нелинейной связи.

Ранговый коэффициент корреляции Спирмена - количественная оценка статистического изучения связи между явлениями, используемая в непараметрических методах. Показатель показывает, как отличается полученная при наблюдении сумма квадратов разностей между рангами от случая отсутствия связи.

-
Метрики на основе каскадной модели поведения

Expected reciprocal rank - ожидаемый обратный ранг документа, на котором остановится пользователь при просмотре страницы. Ожидаемый обратный ранг исходит из идеи, что на практике просмотр часто выполняется последовательно, а вероятность, что пользователь увидит следующий элемент, зависит от релевантности предшествующих.

PFound - представляет бой вероятностную модель поведения пользователя. Пользователь просматривает выдачу сверху вниз. Величина y(q, d) характеризует вероятность того, что пользователь с запросом q удовлетворится документом d. При неуспехе с очередным документом выдачи пользователь разочаруется и уйдёт с вероятностью P out.
